In [1]:
# Module Importations


In [1]:
# Project Module Importations
from GensetSimulation.data_analytics.src.data import cosmos_client_manager

In [3]:
# Constants (i.e. for saving figures)

In [2]:
# Load data from database
client = cosmos_client_manager.intialise_client()
database = cosmos_client_manager.return_database(client)
container = cosmos_client_manager.return_container(database)
item_list = cosmos_client_manager.read_items(container)

Database with id 'BMRS Data' was found, link is dbs/BMRS Data
Container with id 'DataElement' was found, link is dbs/BMRS Data/colls/DataElement
